# Amazon Bedrock AgentCore Memory with Strands AgentCore Memory Session Manager

## Introduction

This notebook demonstrates how to add memory (short-term and long-term) to Strands Agents using AgentCore Memory Session Manager. 

What you will learn:

* Create a shared memory resource that multiple agents can access
* Configure supervisor agent to store user visible conversations to the memory 
* Configure supervisor agent and sub-agents to retrieve recent conversation history from short-term memory as context
* Configure long-term memory strategy and manage data segregation with namespaces
* Congiture agent to retrieve user query related long term memory and use it as context


## Scenario context

In the previous session, you have taken the mortgage assistant built with Strands to Agentcore Runtime, added tools on Agentcore Gateway, authentication and authorisation with Agentcore Idenity. In this session, we continue the journey to make the agent more helpful by adding a memory component using Agentcore Memory.

## Architecture

![agentcore_memory_architecture.png](images/agentcore-memory-strands-session-manager.png)

1. Whenever a user visible conversation is added, whether it is from the user or agent, the conversation will be stored in AgentCore Memory as short-term memory. In Strands `Agents as Tools` setup, all visible conversation will pass through the supervisor agent.
2. Whenever an agent starts, it loads the last 3 conversation from short-term memory as context.
3. A memory extraction module works asynchronously at the background, and use the defined strategy to turn short-term memory into long-term memory.
4. Whenever a user asks a question, the supervisor agent will run a natural language query against the long-term memory and obtain related information as context. 

## Step 0: Environment Setup

In [1]:
import logging
from datetime import datetime
import boto3
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## Step 1: Create Agentcore Memory Resource
First, create Agentcore Memory resource for the mortgage assistant in your AWS account.

In [2]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "mortgage_assistant"
memory_id = None

2025-11-10 14:48:21 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-11-10 14:48:22 - INFO - Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2


In [3]:
# for testing only: if agentcore memory has already been created outside this workshop
# memory_id="mortgage_assistant_20250817122625-BgMl7M34lQ"

In the following code, we create the Agentcore Memory resource.

Notice we create `strategies` here and use the them when creating Agentcore Memory. When `strategies` are provided, Agentcore Memory will use the provided strategies to process short-term memories (conversation histories) and generate long-term memories, such as user facts, user preference or conversation summaries.

If Agentcore Memory is created without `strategies`, it only keeps short-term memories (conversation histories). No long-term memories will be generated.

The Agentcore Memory resource will take a few minutes to create. 

In [4]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Strategies for long term memory
    strategies = [
        {
            StrategyType.SEMANTIC.value: {
                "name": "fact_extractor",
                "description": "Extracts and stores factual information",
                "namespaces": ["mortgage_assistant/{actorId}/facts"],
            },
        },
        {
            StrategyType.SUMMARY.value: {
                "name": "conversation_summary",
                "description": "Captures summaries of conversations",
                "namespaces": ["mortgage_assistant/{actorId}/{sessionId}"],
            },
        },
        {
            StrategyType.USER_PREFERENCE.value: {
                "name": "user_preferences",
                "description": "Captures user preferences and settings",
                "namespaces": ["mortgage_assistant/{actorId}/preferences"],
            },
        },
    ]

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Mortgage Assistant Memory", # Human-readable description
        strategies=strategies,                  # Strategies is for long term memory. For short term memory only, pass in an empty array 
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

Creating Memory...


2025-11-10 14:48:23 - INFO - Created memory: mortgage_assistant-RF59gi983U
2025-11-10 14:48:23 - INFO - Created memory mortgage_assistant-RF59gi983U, waiting for ACTIVE status...
2025-11-10 14:51:08 - INFO - Memory mortgage_assistant-RF59gi983U is now ACTIVE (took 165 seconds)


Memory created successfully with ID: mortgage_assistant-RF59gi983U


Now the agentcore memory is created. You can go to your AWS Console -> Amazon Bedrock Agentcore -> Memory to see the resource. 

Please note, `memory_id` will be used in most API calls to store/retrieve memory. We will store the memory ID in System Manager Parameter Store, so our Agentcore Runtime can fetch this memory ID at runtime to access the Agentcore Memory we created.

In [5]:
ssm = boto3.client("ssm")
param_name = "/app/mortgage_assistant/agentcore/memory_id"

def store_memory_id_in_ssm(param_name: str, memory_id: str):
    ssm.put_parameter(Name=param_name, Value=memory_id, Type="String", Overwrite=True)
    print(f"Stored memory_id in SSM: {param_name}")

In [6]:
store_memory_id_in_ssm(param_name, memory_id)

Stored memory_id in SSM: /app/mortgage_assistant/agentcore/memory_id


## Step 3: Create Multi-Agent Architecture with Strands Agents
Now we can re-create the Agentcore Runtime for the Strands agents as you have done in the last few labs. The execution role for Agentcore Runtime needs access to the Agentcore Memory, and the System Manager Parameter Store to access the Memory ID. This is handled for you in this lab through a project-wide util function.

In [11]:
from utils import create_agentcore_role

agent_name="mortgage_assistant"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

Role already exists


Now we can configure the Agentcore Runtime. 

In [19]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

agentcore_runtime = Runtime()
response = agentcore_runtime.configure(
    entrypoint="option_2_memory_strands_session_manager.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/option_2_memory_strands_session_manager.py, bedrock_agentcore_name=option_2_memory_strands_session_manager
2025-11-10 15:03:56 - INFO - Entrypoint parsed: file=/Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/option_2_memory_strands_session_manager.py, bedrock_agentcore_name=option_2_memory_strands_session_manager
Configuring BedrockAgentCore agent: mortgage_assistant
2025-11-10 15:03:56 - INFO - Configuring BedrockAgentCore agent: mortgage_assistant


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated Dockerfile: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/Dockerfile
2025-11-10 15:03:57 - INFO - Generated Dockerfile: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/Dockerfile
Generated .dockerignore: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/.dockerignore
2025-11-10 15:03:57 - INFO - Generated .dockerignore: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/.dockerignore
Keeping 'mortgage_assistant' as default agent
2025-11-10 15:03:57 - INFO - Keeping 'mortgage_assistant' as default agent
Bedrock AgentCore configured: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/.bedrock_agentcore.yaml
2025-11-10 15:03:57 - INFO - Bedrock AgentCore configured: /Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06

ConfigureResult(config_path=PosixPath('/Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/Dockerfile'), dockerignore_path=PosixPath('/Users/franktan/code/sample-amazon-bedrock-agentcore-prototype-to-production/06-agentcore-memory/.dockerignore'), runtime='None', region='us-west-2', account_id='476559339891', execution_role='arn:aws:iam::476559339891:role/agentcore-mortgage_assistant-role', ecr_repository=None, auto_create_ecr=True)

Now we will launch the AgentCore Runtime resource in your AWS account. It will take a few minutes as the docker image is built in Code Build and deployed to Agentcore Runtime.

In [20]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
2025-11-10 15:03:57 - INFO - 🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
2025-11-10 15:03:57 - INFO -    • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
2025-11-10 15:03:57 - INFO -    • No local Docker required
💡 Available deployment modes:
2025-11-10 15:03:57 - INFO - 💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
2025-11-10 15:03:57 - INFO -    • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
2025-11-10 15:03:57 - INFO -    • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
2025-11-10 15:03:57 - INFO -    • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Sta

✅ Reusing existing ECR repository: 476559339891.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-mortgage_assistant


Reusing existing CodeBuild execution role: arn:aws:iam::476559339891:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-0c50ea17be
2025-11-10 15:03:59 - INFO - Reusing existing CodeBuild execution role: arn:aws:iam::476559339891:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-0c50ea17be
Using .dockerignore with 44 patterns
2025-11-10 15:04:00 - INFO - Using .dockerignore with 44 patterns
Uploaded source to S3: mortgage_assistant/source.zip
2025-11-10 15:04:02 - INFO - Uploaded source to S3: mortgage_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-mortgage_assistant-builder
2025-11-10 15:04:04 - INFO - Updated CodeBuild project: bedrock-agentcore-mortgage_assistant-builder
Starting CodeBuild build (this may take several minutes)...
2025-11-10 15:04:04 - INFO - Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
2025-11-10 15:04:04 - INFO - Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
2025-11-10 15:04:05 - INFO 

Now you can see the Agentcore Runtime resource in your AWS account in Amazon Bedrock Agentcore -> Agent Runtime. But it won't be ready until the status is changed to `READY`

As usual, we check the status of agentcore runtime until it is ready.

In [21]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Retrieved Bedrock AgentCore status for: mortgage_assistant
2025-11-10 15:05:01 - INFO - Retrieved Bedrock AgentCore status for: mortgage_assistant


READY


Once the status is shown as `READY`, the Agentcore Runtime is ready to be invoked.

## Step 4: Invoke the deployed agent

First, let's create a `user_id` and `session_id` for testing. We also create `user_id_for_testing_no_memory` and `session_id_for_testing_no_memory` to test memory isolation. 

In [22]:
import uuid

# An randomly generated fictional user ID. 
# In a real system, replace this with a real user ID in your system
user_id=str(uuid.uuid4())
user_id_for_testing_no_memory=str(uuid.uuid4())

session_id = str(uuid.uuid4())
session_id_for_testing_no_memory = str(uuid.uuid4())

print(f"user_id: {user_id}")
print(f"user_id_for_testing_no_memory: {user_id_for_testing_no_memory}")
print(f"session_id: {session_id}")
print(f"session_id_for_testing_no_memory: {session_id_for_testing_no_memory}")

user_id: 6742bb7b-612b-48d0-bbf4-b213a6176a4b
user_id_for_testing_no_memory: 15f3abcd-996d-4b4d-8570-e4f5c3be3424
session_id: fea63bcb-7bee-492a-be97-491e386cd803
session_id_for_testing_no_memory: 7cfbd3d4-3001-4c96-a7a2-c014f3058f8b


We create a convenient method for printing the reponse messages nicely.

In [23]:
def print_response_text(invoke_response):
    response_bytes=invoke_response['response']
    response_strings = [item.decode('utf-8') for item in response_bytes]
    # print(response_bytes)
    response_combined_string = "".join(response_strings)
    # print(response_combined_string)
    
    response_dict = json.loads(response_combined_string)
    response_text=response_dict["result"]["content"][0]["text"]
    print(response_text)

In [24]:
sample_prompt_1 = "Hey I am John. I'd like to check my existing home loan. My customer id is ABC-123"
sample_prompt_2 = "We are planning to upsize. We are shopping around to see what is avaialble?"
sample_prompt_3 = "Do do banks decide how much it can lend me?"
sample_prompt_4 = "What is the difference between standard variable and fixed rate"
sample_prompt_5 = "What is interest only vs principal and interest?"

Now we call the agent a few times. 

In [25]:
import pprint
import json

invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_1,
    "user_id": user_id
}, session_id=session_id)
# pprint.pprint(invoke_response, width=80, depth=None)
print_response_text(invoke_response)

Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint
2025-11-10 15:05:01 - INFO - Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint


Here's a summary of your existing home loan details:

### Your Mortgage Summary
- **Account Number**: ABC-123
- **Outstanding Principal Balance**: $150,000.00
- **Interest Rate**: 4.5%
- **Maturity Date**: June 30, 2030
- **Payments Remaining**: 72 (6 years)
- **Monthly Payment Amount**: $1,250.00

### Payment Information
- **Last Payment Date**: June 1, 2024
- **Next Payment Due**: July 1, 2024
- **Next Payment Amount**: $1,250.00

Your mortgage is in good standing with regular payments being made. You're on track to fully pay off your loan by 2030.

Is there anything specific about your mortgage you'd like to know more about? I can help with refinancing options, calculating the impact of extra payments, or explaining other aspects of your current mortgage.


In [26]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_2,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint
2025-11-10 15:05:27 - INFO - Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint


Thank you for letting me know about your plans to upsize, John. Based on your existing customer information (ABC-123), I can provide some insights on your mortgage options.

First, I notice that your credit score is 80, which is an important factor in determining your eligibility for different loan products.

### Mortgage Options for Upsizing

Here are some mortgage options to consider for your new home purchase:

**Conventional Mortgages:**
- **30-Year Fixed-Rate Mortgage**: Offers lower monthly payments spread over a longer period
- **15-Year Fixed-Rate Mortgage**: Higher monthly payments but builds equity faster and saves on interest

**Special Mortgage Types:**
- **Jumbo Mortgage**: If you're looking at homes that exceed conforming loan limits

**Financing Approaches:**
- **Cash-Out Refinance**: You could tap into your current home's equity ($150,000 outstanding balance)
- **Rate-and-Term Refinance**: Potentially get better terms before upsizing

**Rate Options:**
- **Fixed-Rate Mo

In [27]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_3,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint
2025-11-10 15:06:14 - INFO - Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint


Banks determine how much they can lend you for a mortgage through a comprehensive evaluation of several key factors. Here's how they make this decision:

### Primary Factors in Mortgage Lending Decisions

1. **Debt-to-Income (DTI) Ratio**
   - This is one of the most critical factors
   - Banks typically prefer a DTI ratio of 43% or less
   - They calculate this by dividing your total monthly debt payments by your gross monthly income
   - This includes your potential mortgage payment plus other debts like car loans, credit cards, etc.

2. **Credit Score and History**
   - Your credit score (which in your case is 80) impacts both approval and interest rates
   - Lenders review your payment history, credit utilization, and length of credit history
   - Higher scores typically qualify for better rates and higher loan amounts

3. **Down Payment**
   - The amount you can put down affects how much you can borrow
   - Larger down payments may allow you to qualify for larger loans
   - Standa

In [28]:
invoke_response = agentcore_runtime.invoke({
    "prompt": sample_prompt_4,
    "user_id": user_id
}, session_id=session_id)
print_response_text(invoke_response)

Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint
2025-11-10 15:06:46 - INFO - Invoking BedrockAgentCore agent 'mortgage_assistant' via cloud endpoint


# Standard Variable Rate vs Fixed Rate Mortgages

## Fixed Rate Mortgages:
- **Interest Rate Stability**: The interest rate remains constant throughout the entire loan term (typically 15 or 30 years)
- **Predictable Payments**: Your monthly payments stay exactly the same for the entire term
- **Budget Certainty**: Makes financial planning easier since you know precisely what your housing costs will be
- **Protection**: Shields you from market fluctuations and rising interest rates
- **Starting Rates**: Generally start slightly higher than initial variable rates
- **Peace of Mind**: You won't have to worry about payment increases if market rates rise

## Standard Variable Rate Mortgages:
- **Fluctuating Rates**: Interest rates can change periodically based on market conditions
- **Variable Payments**: Monthly payments can increase or decrease when rates adjust
- **Lower Initial Rates**: Often starts with lower rates compared to fixed-rate options
- **Potential Savings**: Beneficial if i

## Step 4: Check memory

After calling the agents a few times, we can now check the memory. We use two ways to check the memory.

1. We will check the memory with the user_id and session_id with Bedrock API. This allows us to check the stored short-term and long-term memory.
2. We will check the CloudWatch log to see the exact memory operations at the lifecycle events in supervisor and sub-agents. This helps us to adapt our memory management to suit our use case.

### 4.1 Check short-term memory
First, we create a convenient method for listing out the last K conversation turns from the short-term memory.

In [52]:
# Check what's stored in memory
def list_last_k_turns(memory_id, actor_id, session_id, k):
    print("=== Memory Contents ===")
    print(f"actor_id: {actor_id}")
    print(f"session_id: {session_id}")
    
    recent_turns = client.get_last_k_turns(
        memory_id=memory_id,
        actor_id=actor_id,
        session_id=session_id,
        k=k # Adjust k to see more or fewer turns
        # branch_name="main"
    )
    
    for i, turn in enumerate(recent_turns, 1):
        print(f"Turn {i}:")
        for message in turn:
            role = message['role']
            content = json.loads(message['content']['text'])
            # content = message['content']['text'][:400] + " <🖨️ ... omitted for printing only ... 🖨️>" if len(message['content']['text']) > 400 else message['content']['text']
            print(f"  {role}: {json.dumps(content, indent=4)}")
        print()

Using a valid `user_id` and `session_id`, we expect to see the recent conversation history.

In [53]:
list_last_k_turns(memory_id, f"{user_id}", session_id, 20)

=== Memory Contents ===
actor_id: 6742bb7b-612b-48d0-bbf4-b213a6176a4b
session_id: fea63bcb-7bee-492a-be97-491e386cd803


2025-11-10 16:12:20 - INFO - Retrieved total of 22 events


Turn 1:
  ASSISTANT: {
    "message": {
        "role": "assistant",
        "content": [
            {
                "text": "# Standard Variable Rate vs Fixed Rate Mortgages\n\n## Fixed Rate Mortgages:\n- **Interest Rate Stability**: The interest rate remains constant throughout the entire loan term (typically 15 or 30 years)\n- **Predictable Payments**: Your monthly payments stay exactly the same for the entire term\n- **Budget Certainty**: Makes financial planning easier since you know precisely what your housing costs will be\n- **Protection**: Shields you from market fluctuations and rising interest rates\n- **Starting Rates**: Generally start slightly higher than initial variable rates\n- **Peace of Mind**: You won't have to worry about payment increases if market rates rise\n\n## Standard Variable Rate Mortgages:\n- **Fluctuating Rates**: Interest rates can change periodically based on market conditions\n- **Variable Payments**: Monthly payments can increase or decrease when 

Next, check the memory with a different user_id `user_id_for_testing_no_memory` which does not exist. We expect to see nothing.

In [54]:
list_last_k_turns(memory_id, f"{user_id_for_testing_no_memory}-supervisor", session_id, 10)

=== Memory Contents ===
actor_id: 15f3abcd-996d-4b4d-8570-e4f5c3be3424-supervisor
session_id: fea63bcb-7bee-492a-be97-491e386cd803


2025-11-10 16:13:34 - INFO - Retrieved total of 0 events


Check the memory with the same `user_id`, but a different session_id `session_id_for_testing_no_memory`. Again, we expect to see NO messages.

In [55]:
list_last_k_turns(memory_id, f"{user_id}", session_id_for_testing_no_memory, 10)

2025-11-10 16:13:35 - INFO - Retrieved total of 0 events


=== Memory Contents ===
actor_id: 6742bb7b-612b-48d0-bbf4-b213a6176a4b
session_id: 7cfbd3d4-3001-4c96-a7a2-c014f3058f8b


#### CloudWatch Log
Let's find the the log file in CloudWatch. You should see the short-term memory loaded when the agent is invoked. As we rely on AgentCore Memory Session Manager's default behaviour, we do not see the full message in the log.

![option-2-short-term-memory-retrieval.png](images/option-2-short-term-memory-retrieval.png)

You should also see memory created event as the current conversation is stored to memory.

![option-2-create-memory-event.png](images/option-2-create-memory-event.png)

### 4.2 Check long-term memory
Again, we will first use `list_memory_records` method to see the long-term memory.

In [33]:
agentcore_client = boto3.client("bedrock-agentcore")

print("----------")
print("Long term memory - facts:")
response_facts = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/facts",
    maxResults=10
)
# pprint.pprint(response_facts, width=80, depth=None)
for list_memory_record in response_facts["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - preferences:")
response_preferences = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/preferences",
    maxResults=10
)
# pprint.pprint(response_preferences, width=80, depth=None)
for list_memory_record in response_preferences["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

print("Long term memory - summary:")
response_summary = agentcore_client.list_memory_records(
    memoryId=memory_id,
    namespace=f"mortgage_assistant/{user_id}/{session_id}",
    maxResults=10
)
# pprint.pprint(response_summary, width=80, depth=None)
for list_memory_record in response_summary["memoryRecordSummaries"]:
    print(f"Content: {list_memory_record['content']['text']}")
print("----------")

----------
Long term memory - facts:
Content: John's current mortgage has an interest rate of 4.5%.
Content: John's mortgage maturity date is June 30, 2030.
Content: John's last mortgage payment was made on June 1, 2024.
Content: John's next mortgage payment is due on July 1, 2024.
Content: John has a mortgage application in progress that is missing employment information.
Content: John is planning to upsize his home.
Content: John's customer ID is ABC-123.
Content: John has an existing home loan with an outstanding principal balance of $150,000.00.
Content: John has 72 payments remaining on his mortgage.
Content: John's monthly mortgage payment is $1,250.00.
----------
Long term memory - preferences:
Content: {"context":"The user explicitly mentioned that they are planning to upsize their home.","preference":"Planning to upsize home","categories":["real estate","housing"]}
Content: {"context":"The user mentioned they are shopping around to see what is available, indicating they are in

Alternatively, we can also use `retrieve_memories` method to retrieve long-term memory.

In [34]:
memory_client=MemoryClient(region_name=region)
print("----------")
print("Long term memory - facts:")
facts_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/facts", query="all", top_k=10
        )
# to see the raw facts memory records
# print(facts_memory)
# print the content
for memory in  facts_memory:
    print(memory["content"]["text"])

print("----------")
print("Long term memory - preferences:")
prefs_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/preferences", query="all", top_k=10
        )
# print the content
for memory in  prefs_memory:
    memory_content = memory["content"]["text"]
    memory_content_json = json.loads(memory_content)
    print(json.dumps(memory_content_json, indent=4))

print("----------")
print("Long term memory - summaries:")
summaries_memory=memory_client.retrieve_memories(
            memory_id=memory_id, namespace=f"mortgage_assistant/{user_id}/{session_id}", query="all", top_k=10
        )
# print the content
for memory in  summaries_memory:
    print(memory["content"]["text"])
print("----------")

2025-11-10 15:07:09 - INFO - Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2


----------
Long term memory - facts:


2025-11-10 15:07:09 - INFO - Retrieved 10 memories from namespace: mortgage_assistant/6742bb7b-612b-48d0-bbf4-b213a6176a4b/facts


John's customer ID is ABC-123.
John's last mortgage payment was made on June 1, 2024.
John has 72 payments remaining on his mortgage.
John's next mortgage payment is due on July 1, 2024.
John's monthly mortgage payment is $1,250.00.
John has an existing home loan with an outstanding principal balance of $150,000.00.
John's current mortgage has an interest rate of 4.5%.
John has a mortgage application in progress that is missing employment information.
John is planning to upsize his home.
John's mortgage maturity date is June 30, 2030.
----------
Long term memory - preferences:


2025-11-10 15:07:10 - INFO - Retrieved 2 memories from namespace: mortgage_assistant/6742bb7b-612b-48d0-bbf4-b213a6176a4b/preferences


{
    "context": "The user mentioned they are shopping around to see what is available, indicating they are in the research phase of home buying.",
    "preference": "Currently shopping around for home options",
    "categories": [
        "real estate",
        "housing",
        "shopping behavior"
    ]
}
{
    "context": "The user explicitly mentioned that they are planning to upsize their home.",
    "preference": "Planning to upsize home",
    "categories": [
        "real estate",
        "housing"
    ]
}
----------
Long term memory - summaries:


2025-11-10 15:07:10 - INFO - Retrieved 1 memories from namespace: mortgage_assistant/6742bb7b-612b-48d0-bbf4-b213a6176a4b/fea63bcb-7bee-492a-be97-491e386cd803


    <topic name="Customer Information">
        Customer named John with customer ID ABC-123 is inquiring about his existing home loan and exploring options to upsize his home.
    </topic>
    <topic name="Existing Mortgage Details">
        John&#39;s current mortgage has an outstanding principal balance of $150,000.00 with a 4.5% interest rate. The loan matures on June 30, 2030, with 72 payments remaining (6 years). His monthly payment is $1,250.00. His last payment was on June 1, 2024, with the next payment of $1,250.00 due on July 1, 2024. The mortgage is in good standing with regular payments.
    </topic>
    <topic name="Upsize Plans">
        John is planning to upsize his home and is shopping around to see what options are available. He already has a mortgage application in progress but it&#39;s missing employment information. He needs to either complete the current application by providing employment details or adjust the application to reflect his upsizing plans.
    </topi

#### CloudWatch
Let's find the the log file in CloudWatch. You should see long term memory loaded from the namespaces we defined.

![option-2-long-term-memory-retrieval.png](images/option-2-long-term-memory-retrieval.png)

#### Summary 
You have added memory to your multi-agent Mortgage Assistant. 

You can now move to the next session.